In [0]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient

In [0]:
#Permite enviar credenciales por un canal seguro
from msrest.authentication import ApiKeyCredentials

In [0]:
proyectoId = "e157a59f-6b6e-40da-9e16-dfd525b4c7b0"
clave = "65724f8c7c564df98eeee5080bdd5e4b"
endpoint = "https://customvisionlgpr-prediction.cognitiveservices.azure.com/"

In [0]:
modelo = "MODELO_TORNILLOS_1"

In [0]:
credenciales = ApiKeyCredentials(in_headers = {"Prediction-key" : clave})


In [0]:
servicio = CustomVisionPredictionClient(endpoint, credenciales)

In [0]:
%fs cp dbfs:///mnt/azure/data/TORNILLO.png file:///tmp

res0: Boolean = true

In [0]:
with open("/tmp/TORNILLO.png","rb") as imagen:
  resultado = servicio.classify_image(
    proyectoId,
    modelo,
    imagen.read()
  ) 

In [0]:
resultado.predictions[1].tag_name

Out[8]: 'TORNILLO_OK'

In [0]:
resultado.predictions[1].probability

Out[9]: 0.44499582

In [0]:
tags = []
tagsProbabilidad = []

In [0]:
for prediccion in resultado.predictions:
  tags.append(prediccion.tag_name)
  tagsProbabilidad.append(prediccion.probability)
  print(prediccion.tag_name + ": "+ str(prediccion.probability))

TORNILLO_DEFECTUOSO: 0.5550041
TORNILLO_OK: 0.44499582


In [0]:
tags

Out[12]: ['TORNILLO_DEFECTUOSO', 'TORNILLO_OK']

In [0]:
tagsProbabilidad

Out[13]: [0.5550041, 0.44499582]

In [0]:
import numpy as np


In [0]:
numpyTagsProbabilidad = np.array(tagsProbabilidad)

In [0]:
numpyTagsProbabilidad

Out[16]: array([0.5550041 , 0.44499582])

In [0]:
indiceProbabilidadMayor = np.argmax(numpyTagsProbabilidad)

In [0]:
indiceProbabilidadMayor

Out[22]: 0

In [0]:
tagPredicho = tags[indiceProbabilidadMayor]

In [0]:
tagPredicho

Out[26]: 'TORNILLO_DEFECTUOSO'

In [0]:
#Funcion de prediccion visual
def prediccionVisual(rutaImagen):
  #Abrimos la imagen en modo binario
  with open(rutaImagen,"rb") as imagen:
    resultado = servicio.classify_image(
    proyectoId,
    modelo,
    imagen.read()
    )
  #Acumulamos las probabilidades de cada categoria predicha
  for prediccion in resultado.predictions:
    tags.append(prediccion.tag_name)
    tagsProbabilidad.append(prediccion.probability)
  #Convertimos el array a un array NUMPY para poder usar los utilitarios en él
  numpyTagsProbabilidad = np.array(tagsProbabilidad)

  #Seleccionamos el indice con la mayor probabilidad
  indiceProbabilidadMayor = np.argmax(numpyTagsProbabilidad)

  #Con el indice de mayor probabilidad, obtenemos el nombre del tag
  tagPredicho = tags[indiceProbabilidadMayor]

  #Retornamos la predicción
  return tagPredicho


In [0]:
%fs cp dbfs:///mnt/azure/data/TORNILLO.png file:///tmp

res1: Boolean = true

In [0]:
tagPredicho = prediccionVisual("/tmp/TORNILLO.png")

In [0]:
tagPredicho

Out[31]: 'TORNILLO_DEFECTUOSO'